In [8]:
import sys
import os

sys.path.append(os.path.join(os.getcwd(), "..", ".."))
import config, utils


import polars as pl

client = utils.get_hdfs_client()

with client.read(os.path.join(config.FILTERED_DATA_DIR, "ball_to_ball.csv")) as reader:
    df = pl.read_csv(reader)

[2024-11-15T21:27:35.540+0530] {base.py:84} INFO - Retrieving connection 'webhdfs_default'
[2024-11-15T21:27:35.543+0530] {webhdfs.py:82} INFO - Trying to connect to 192.168.245.142:9870
[2024-11-15T21:27:35.546+0530] {webhdfs.py:86} INFO - Trying namenode 192.168.245.142
[2024-11-15T21:27:35.548+0530] {client.py:192} INFO - Instantiated <InsecureClient(url='http://192.168.245.142:9870/')>.
[2024-11-15T21:27:35.550+0530] {client.py:320} INFO - Fetching status for '/'.
[2024-11-15T21:27:35.560+0530] {webhdfs.py:96} INFO - Using namenode 192.168.245.142 for hook
[2024-11-15T21:27:35.562+0530] {client.py:724} INFO - Reading file '/usr/ravi/t20/data/4_filteredData/ball_to_ball.csv'.


HdfsError: Path is not a file: /usr/ravi/t20/data/4_filteredData/ball_to_ball.csv
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:90)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:156)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:2124)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:770)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.getBlockLocations(ClientNamenodeProtocolServerSideTranslatorPB.java:460)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlockingMethod(ClientNamenodeProtocolProtos.java)
	at org.apache.hadoop.ipc.ProtobufRpcEngine2$Server$ProtoBufRpcInvoker.call(ProtobufRpcEngine2.java:621)
	at org.apache.hadoop.ipc.P

In [15]:
balltoball.count(),balltoball[["match_id","flip"]].distinct().count(),player_stats.count(),player_stats[["match_id","flip"]].distinct().count(),team12_stats.count(),team12_stats[["match_id","flip"]].distinct().count()

(525498, 2270, 49940, 2270, 2270, 2270)

In [10]:
print(list(team12_stats_list[30][0].values())[2:], list(player_stats_list[30][0].values())[2:], list(balltoball_list[30][0].values())[2:])

[0, 1, 0, 0, 0.0, 16.6, 0.0, 1, 1, 1, 1.0, 22.75, 8.38] [1, 2, 40.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0] [1, 0.1, 3, 0, 0, 1]


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

# list(team12_stats_list[30][0].values())[2:]
# Create a custom Dataset
class CricketDataset(Dataset):
    def __init__(self, team_stats_list, player_stats_list, ball_stats_list, labels):
        self.team_stats_list = team_stats_list
        self.player_stats_list = player_stats_list
        self.ball_stats_list = ball_stats_list
        self.labels = labels
        self.len = len(labels)

    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        team_input = torch.tensor(self.team_stats_list[index], dtype=torch.float32)
        team_input = team_input.squeeze()
        player_input = torch.tensor(self.player_stats_list[index], dtype=torch.float32)
        ball_stats = torch.tensor(self.ball_stats_list[index], dtype=torch.float32)
        ball_input = ball_stats
        label = torch.tensor(self.labels[index], dtype=torch.float32)
        return team_input, player_input, ball_input, label
    
# Define a collate function to handle variable-length sequences
def collate_fn(batch):
    team_inputs = []
    player_inputs = []
    ball_inputs = []
    labels = []
    ball_lengths = []

    for team_input, player_input, ball_input, label in batch:
        team_inputs.append(team_input)
        player_inputs.append(player_input)
        ball_inputs.append(ball_input)
        labels.append(label)
        ball_lengths.append(ball_input.shape[0])

    max_seq_len = max(ball_lengths)
    padded_ball_inputs = torch.zeros(len(ball_inputs), max_seq_len, ball_inputs[0].shape[1])
    for i, ball_input in enumerate(ball_inputs):
        seq_len = ball_input.shape[0]
        padded_ball_inputs[i, :seq_len, :] = ball_input

    team_inputs = torch.stack(team_inputs)
    player_inputs = torch.stack(player_inputs)
    labels = torch.tensor(labels, dtype=torch.float32)
    return team_inputs, player_inputs, padded_ball_inputs, labels, ball_lengths